# H&M Personalized Fashion Recommendations

## Data Preprocessing

Required external libraries:

In [1]:
import pandas as pd

Read data:

In [12]:
articles_data = pd.read_csv('articles.csv')
customers_data = pd.read_csv('customers.csv')

## Features Selection